In [ ]:
from TwitterSearch import *
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['covid', 'vaccination']) # let's define all words we would like to have a look for
    tso.set_language('en') # we want to see German tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    tso.set_geocode(-37.840935, 144.946457, 100)
    tso.set_count(10)

    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = "Ozup2OAf8pHZhha38AELtzswf",
        consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV",
        access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh",
        access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"
     )

     # this is where the fun actually starts :)
    for tweet in ts.search_tweets_iterable(tso):
        print(tweet)
        print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

{'created_at': 'Sat May 08 00:08:38 +0000 2021', 'id': 1390820905589239808, 'id_str': '1390820905589239808', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 39191308, 'id_str': '39191308', 'name': 'cubbie |🌈| CB Mako', 'screen_name': 'cubbieberry', 'location': 'stolen lands of Kulin Nation', 'description': "♡ #slow • late-diagnosis ASD▪︎non-binary ▪︎\n'Growing Up Disabled In Australia' anthology (Feb 2021)▪︎'Collisions' Liminal Fictional Prize anthology (Nov 2020)", 'url': 'https://t.co/71l1M3XowN', 'entities': {'url': {'urls': [{

{'created_at': 'Fri May 07 23:47:49 +0000 2021', 'id': 1390815667255865347, 'id_str': '1390815667255865347', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2425245776, 'id_str': '2425245776', 'name': 'Dr Claire Wylie', 'screen_name': 'DrCEWylie', 'location': 'Melbourne, Victoria', 'description': '🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f | Veterinarian | Epidemiologist | Human Clinical Toxicology | Evidence Based Veterinary Medicine | Horses 🐴', 'url': 'https://t.co/FHrHJPwTxx', 'entities': {'url': {'urls': [{'u

{'created_at': 'Fri May 07 23:36:27 +0000 2021', 'id': 1390812804706304000, 'id_str': '1390812804706304000', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 343096504, 'id_str': '343096504', 'name': '💧 Blue-tongued lizard', 'screen_name': 'darylgibson', 'location': 'Manning Point', 'description': "jackeroo decky waiter model pro Harvard Mol Cell Biol Research Tech Radiological Health Imaging Dosimetry cancer research Physicist MSc ret'd, not wanted in oz", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_c

{'created_at': 'Fri May 07 23:27:55 +0000 2021', 'id': 1390810657512038401, 'id_str': '1390810657512038401', 'text': 'RT @VictorianCHO: At current vaccination coverage, the spread of COVID-19 isn’t even slowed, let alone stopped. That’s why I’m glad Victori…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1664093738, 'id_str': '1664093738', 'name': 'Lesley Howard', 'screen_name': 'adropex', 'location': 'Melbourne', 'description': 'Language matters and statistics don’t lie. Only people do. https://t.co/TRFPL9BCfZ (Applied Statistics) and other stuff.', 'url': None, 'entities': {'description': {'urls': [{'url': 'https://t.co/TRFPL9BCfZ', 'expanded_url': 'http://M.Sc', 'display_url'

{'created_at': 'Fri May 07 21:53:54 +0000 2021', 'id': 1390786996881166341, 'id_str': '1390786996881166341', 'text': 'Impact and effectiveness of mRNA BNT162b2 vaccine against SARS-CoV-2 infections and COVID-19 cases, hospitalization… https://t.co/55WWwuhiUu', 'truncated': True, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 122878913, 'id_str': '122878913', 'name': 'Rasik Vajpayee', 'screen_name': 'rasikv', 'location': 'Melbourne', 'description': 'Every story can have ten different truths! लोग बदलते नहीं बेनक़ाब होते हैं 🙂', 'url': 'https://t.co/pUsUDYO0j7', 'entities': {'url': {'urls': [{'url': 'https://t.co/pUsUDYO0j7', 'expanded_url': 'http://rasikv.blogspot.com.au/', 'display_url': 'rasikv.blogspot.com.au',

...

{'created_at': 'Tue May 04 10:47:52 +0000 2021', 'id': 1389532220096929796, 'id_str': '1389532220096929796', 'text': 'First step towards Mass vaccination by @MaxHealthcare is exactly what was needed to protect Delhi from COVID. Take a bow!', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1352983675902676994, 'id_str': '1352983675902676994', 'name': 'Ziddi boy', 'screen_name': 'Ziddiboy__', 'location': 'Melbourne, Australia', 'description': "I say luck is when an opportunity comes along and you're prepared for it.", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 1050, 'friends_count': 3000, 'listed_count': 0, 'created_at': 'Sat Jan 

{'created_at': 'Tue May 04 08:54:57 +0000 2021', 'id': 1389503804211568640, 'id_str': '1389503804211568640', 'text': "RT @behrooz_hm: The effect of Victoria's mass vaccination hubs is now clearly visible in the data as the state records the highest number o…", 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1241679821123358720, 'id_str': '1241679821123358720', 'name': 'Richard Wazny', 'screen_name': 'WaznyRichard', 'location': 'Woiworong Land, Melbourne', 'description': 'Mathematician, IT Consultant/trouble shooter now out smelling the roses', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 490, 'friends_count': 2272, 'listed_count':

{'created_at': 'Tue May 04 07:58:22 +0000 2021', 'id': 1389489565765836801, 'id_str': '1389489565765836801', 'text': "RT @euan_wallace: Brilliant response from Victoria's over 50s!!! Melbourne's COVID-19 mass vaccination centres swamped.  Brilliant\nhttps://…", 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 735074546038886400, 'id_str': '735074546038886400', 'name': 'Shu-Yi Soong', 'screen_name': 'ShuSoong', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 22, 'friends_count': 198, 'listed_count': 0, 'created_at': 'Tue May 24 11:46:43 +0000 2016', 'favourites_count': 1617, 'utc_offset': None, 'time_zo

{'created_at': 'Tue May 04 06:04:08 +0000 2021', 'id': 1389460815984566273, 'id_str': '1389460815984566273', 'text': 'RT @StVincentsMelb: If you’re aged 50 and over, you can now receive the COVID-19 vaccine at our high-volume vaccination centre at The Royal…', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 117315665, 'id_str': '117315665', 'name': 'ANMF (Vic Branch)', 'screen_name': 'anmfvic', 'location': 'Victoria', 'description': 'ANMF (Vic Branch) represents Victorian nurses, midwives and carers.  RTs =\\= endorsements. \nAuthorised by L Fitzpatrick, Secretary, ANMF (Vic Branch), Melbourne', 'url': 'https://t.co/AsuOESU4UJ', 'entities': {'url': {'urls': [{'url': 'https://t.co/AsuOESU4UJ',

{'created_at': 'Tue May 04 03:22:01 +0000 2021', 'id': 1389420018857771008, 'id_str': '1389420018857771008', 'text': 'RT @maxwalden_: New Zealand’s Covid vaccination ad campaign https://t.co/CiFNZfsWl7', 'truncated': False, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 10482452, 'id_str': '10482452', 'name': 'Paul Wiggins', 'screen_name': 'paulwiggins', 'location': 'Homebush NSW', 'description': 'Retired editor and designer.', 'url': 'https://t.co/pqRLpS1n4s', 'entities': {'url': {'urls': [{'url': 'https://t.co/pqRLpS1n4s', 'expanded_url': 'https://www.flickr.com/photos/paulwiggins', 'display_url': 'flickr.com/photos/paulwig…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False

In [2]:
db = couch.create('test')

ConnectionRefusedError: [Errno 61] Connection refused